# Import libraries

In [1]:
import os
import tweepy
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from pymongo import MongoClient
from datetime import datetime


f:\Language Courses Videos\Machine Learning\Complete Data Science, Machine Learning, Deep Learning, NLP Bootcamp 2024-10\python\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



# 1. Twitter API setup

In [2]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAFXk4AEAAAAA40%2B14zrsFGOzNZQeV%2Bv1%2FTtwxXg%3DRwENnDKJPI4msY0vZdIJG7q8ZeZjWtMIoELPsjbIW5kK8yC8lG"
  # <-- replace with your token

if not BEARER_TOKEN:
    raise ValueError("Bearer token not found. Please set TWITTER_BEARER_TOKEN environment variable.")
  
client = tweepy.Client(bearer_token=BEARER_TOKEN)


Cell 2: Import libraries and load Bearer Token

# 2. Define Tweet Preprocessing function

In [3]:

def preprocess_tweet(text):
    text = re.sub(r'http\S+|@\S+|#\S+', '', text)  # remove urls, mentions, hashtags
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)    # remove special characters
    return text.lower().strip()


# 3. Load BERT model & Sentiment model classifier


In [4]:
MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
sentiment_classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

def classify_sentiment(text):
    result = sentiment_classifier(text)[0]
    label = result['label']   # e.g. "4 stars"
    score = result['score']

    if "1" in label or "2" in label:
        sentiment = "Negative"
    elif "3" in label:
        sentiment = "Neutral"
    else:
        sentiment = "Positive"

    return sentiment, score


Device set to use cpu


# 4. Setup MongoDB connection and save function


In [5]:
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client.twitter_sentiment
tweets_collection = db.tweets

def save_to_db(tweet, sentiment, score):
    doc = {
        'tweet_id': tweet.id,
        'text': tweet.text,
        'clean_text': preprocess_tweet(tweet.text),
        'sentiment': sentiment,
        'score': score,
        'created_at': tweet.created_at,
        'lang': tweet.lang,
        'inserted_at': datetime.utcnow()
    }
    tweets_collection.insert_one(doc)


# 5. Fetch tweets (instead of streaming)


In [7]:
query = "#AI lang:en -is:retweet"  # hashtag + English + no retweets
tweets = client.search_recent_tweets(query=query,
                                     tweet_fields=["created_at", "lang"],
                                     max_results=10)


# 6. Process and save

In [8]:
if tweets.data:
    for tweet in tweets.data:
        clean = preprocess_tweet(tweet.text)
        sentiment, score = classify_sentiment(clean)
        save_to_db(tweet, sentiment, score)
        print(f"[{tweet.created_at}] {sentiment} ({score:.2f}): {tweet.text}")
else:
    print("No tweets found.")

C:\Users\Dell\AppData\Local\Temp\ipykernel_23852\485228897.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'inserted_at': datetime.utcnow()


[2025-09-16 09:59:47+00:00] Positive (0.53): Our datasets are a massive collection of high-quality, multimodal data, including: Text, Multimodal, Global, and Logical Reasoning.
#AI #data #SupplyChain https://t.co/U8cAOx01Mi
[2025-09-16 09:59:42+00:00] Negative (0.30): 🚀 AI made headlines today:

🚗 NVIDIA’s new AIBOX puts large AI models inside cars.

👓 Meta’s AI glasses face supply chain risks tied to China.

🌍 https://t.co/9X1XmtyFYv &amp; Expedia brace for disruption from AI travel agents.

#AI #Meta #Travel #Business #Nvidia https://t.co/GPzVFJmswZ
[2025-09-16 09:59:39+00:00] Negative (0.45): I talk about how much fewer revenues are coming out of the #AI system (and no profits) than investments are going in. The pop is getting close particularly with the unethical lease backs of Nvidia chips.  https://t.co/8YhJl3FuhB
[2025-09-16 09:59:35+00:00] Positive (0.49): 🏡 Buying a home just got smarter 🔗 reAlpha expands Claire, an AI concierge that captures your budget, lifestyle &amp; locat

Cell 7: Define tweet streaming client class


In [ ]:
class MyStream(tweepy.StreamingClient):
    def on_tweet(self, tweet):
        clean = preprocess_tweet(tweet.text)
        sentiment, score = classify_sentiment(clean)
        save_to_db(tweet, sentiment, score)
        print(f"[{tweet.created_at}] {sentiment} ({score:.2f}): {tweet.text}")

Cell 8: Start streaming tweets for hashtag #AI


In [ ]:
stream = MyStream(BEARER_TOKEN)

# Delete existing rules to avoid duplicates
rules = stream.get_rules()
if rules.data:
    stream.delete_rules([rule.id for rule in rules.data])

# Add hashtag rule
stream.add_rules(tweepy.StreamRule("#AI"))

# Start streaming; specify tweet fields
stream.filter(tweet_fields=["created_at", "lang", "geo"])